In [19]:
import dill
import pandas as pd

In [41]:
#код, который использовался для объединения бачей предсказаний
with open('cosine1_offline.dill', "rb") as f:
    p1 = dill.load(f)
with open('cosine2_offline.dill', "rb") as f:
    p2 = dill.load(f)
with open('cosine3_offline.dill', "rb") as f:
    p3 = dill.load(f)
with open('cosine4_offline.dill', "rb") as f:
    p4 = dill.load(f)

all_data = {}
all_data.update(p1)
all_data.update(p2)
all_data.update(p3)
all_data.update(p4)

print(len(list(all_data.items())))

with open("cosine_offline.dill", "wb") as f:
    dill.dump(all_data, f, -1)

840196


In [67]:
#загрузка всех необходимых данных
users = pd.read_csv('service/data/users.csv')
users = list(users['user_id'])

with open('bm25_offline.dill', "rb") as f:
    bm25_data = dill.load(f)

with open('tfidf_offline.dill', "rb") as f:
    tfidf_data = dill.load(f)
    
with open('cosine_offline.dill', "rb") as f:
    cosine_data = dill.load(f)

Проведем небольшой анализ полученных рекомендаций

In [68]:
#Проверим сколько пустых рекомендаций есть в предсказаниях
tfidf_empty = sum(x == [] for x in tfidf_data.values())
bm25_empty = sum(x == [] for x in bm25_data.values())
cosine_empty = sum(x == [] for x in tfidf_data.values())

print("tfidf number of empty recs: {}".format(tfidf_empty))
print("bm25 number of empty recs: {}".format(bm25_empty))
print("cosine number of empty recs: {}\n".format(cosine_empty))

#Проверим сколько есть не пустых рекомендаций, но меньше 10
tfidf_nonempty = sum((len(x) >= 1 and len(x) < 10) for x in tfidf_data.values())
bm25_nonempty = sum((len(x) >= 1 and len(x) < 10) for x in bm25_data.values())
cosine_nonempty = sum((len(x) >= 1 and len(x) < 10) for x in tfidf_data.values())

print("tfidf number of non empty, but <10 recs: {}".format(tfidf_nonempty))
print("bm25 number of non empty, but <10 recs: {}".format(bm25_nonempty))
print("cosine number of non empty, but <10 recs: {}".format(cosine_nonempty))

tfidf number of empty recs: 384091
bm25 number of empty recs: 641291
cosine number of empty recs: 384091

tfidf number of non empty, but <10 recs: 215743
bm25 number of non empty, but <10 recs: 4940
cosine number of non empty, but <10 recs: 215743


Пронализировав ситуацию на лидерборде (самый высокий скор у bm25 моделей) и результаты оффлайн рекомендаций (анализ кол-ва из блока выше), было принято решение использовать такой алгоритм для объединения - Сначала берем предсказания bm25, если их меньше 10, то берем tfidf, если и их меньше, то cosine, если и их меньше, то предлагаем популярное.

P.S. в процессе прогона, оказалось, что при вычислении оффлайна один id потерялся, из-за этого был добавлен try/catch и пользователю присвоено популярное 

In [91]:
counts = [0] * 4 #bm25, tfidf, cosine, popular
final_data = {}

for user in users:
    try:
        recs = bm25_data.get(user)
        if (len(recs) < 10):
            recs = tfidf_data.get(user)
            if (len(recs) < 10):
                recs = cosine_data.get(user)
                if (len(recs) < 10):
                    recs = [10440, 15297, 9728, 13865, 3734, 12192, 4151, 11863, 7793, 7829]
                    counts[3] += 1
                else:
                    counts[2] += 1
            else:
                counts[1] += 1
        else:
            counts[0] += 1
        final_data.update({user: recs})
    except:
        final_data.update({user: [10440, 15297, 9728, 13865, 3734, 12192, 4151, 11863, 7793, 7829]})
        counts[3] += 1

print("bm25 count: {}".format(counts[0]))
print("tfidf count: {}".format(counts[1]))
print("cosine count: {}".format(counts[2]))
print("popular count: {}".format(counts[3]))

with open("all_alg1.dill", "wb") as f:
    dill.dump(final_data, f, -1)

bm25 count: 193965
tfidf count: 161110
cosine count: 45224
popular count: 439898


Ради интереса немного изменим алгоритм, и, перед тем как предложить популярное, попробуем объеденить рекомендации всех 3 моделей

In [87]:
counts = [0] * 5 #bm25, tfidf, cosine, sum, popular
final_data = {}

for user in users:
    try:
        recs_bm25 = bm25_data.get(user)
        if len(recs_bm25) < 10:
            recs_tfidf = tfidf_data.get(user)
            if len(recs_tfidf) < 10:
                recs_cosine = cosine_data.get(user)
                if len(recs_cosine) < 10:
                    recs_all = list(set(recs_bm25) | set(recs_tfidf) | set(recs_cosine))[:10]
                    if len(recs_all) < 10:
                        recs = [10440, 15297, 9728, 13865, 3734, 12192, 4151, 11863, 7793, 7829]
                        counts[4] += 1
                        final_data.update({user: recs})
                    else:
                        counts[3] += 1
                        final_data.update({user: recs_all})
                else:
                    counts[2] += 1
                    final_data.update({user: recs_cosine})
            else:
                counts[1] += 1
                final_data.update({user: recs_tfidf})
        else:
            counts[0] += 1
            final_data.update({user: recs_bm25})
    except:
            final_data.update({user: [10440, 15297, 9728, 13865, 3734, 12192, 4151, 11863, 7793, 7829]})
            counts[4] += 1


print("bm25 count: {}".format(counts[0]))
print("tfidf count: {}".format(counts[1]))
print("cosine count: {}".format(counts[2]))
print("sum count: {}".format(counts[3]))
print("popular count: {}".format(counts[4]))

with open("all_alg2.dill", "wb") as f:
    dill.dump(final_data, f, -1)

bm25 count: 193965
tfidf count: 161110
cosine count: 45224
sum count: 18152
popular count: 421746
